In [ ]:
import json
import requests
import csv

#讀取資料
URL_api = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization=rdec-key-123-45678-011121314'
resp = requests.get(URL_api)
data = resp.json()
data = data['records']['location']
row_data = []
for item in data :
    lat = item['lat']
    lon = item['lon']
    locationName = item['locationName']
    stationId = item['stationId']
    obsTime = item['time']['obsTime']
    ELE = item['weatherElement'][0]['elementValue']
    WDIR = item['weatherElement'][1]['elementValue']
    WDSD = item['weatherElement'][2]['elementValue']
    TEMP = item['weatherElement'][3]['elementValue']
    HUMD = item['weatherElement'][4]['elementValue']
    PRES = item['weatherElement'][5]['elementValue']
    dic = {}
    dic['lat'] = lat
    dic['lon'] = lon
    dic['locationName'] = locationName
    dic['stationId'] = stationId
    dic['obsTime'] = obsTime
    dic['ELE'] = ELE
    dic['WDIR'] = WDIR
    dic['WDSD'] = WDSD
    dic['TEMP'] = TEMP
    dic['HUMD'] = HUMD
    dic['PRES'] = PRES
    row_data.append(dic)
    print(dic)

#寫入csv檔
headers = ['lat','lon','locationName','stationId','obsTime','ELE','WDIR','WDSD','TEMP','HUMD','PRES']

with open('weather.csv','w') as outfile :
    dic_writer = csv.DictWriter(outfile,headers)
    dic_writer.writeheader()
    dic_writer.writerows(row_data)
    


In [ ]:
#自動進行程式
import json
import requests
import csv
from apscheduler.schedulers.blocking import BlockingScheduler


def get_data(URL_api) :
    resp = requests.get(URL_api)
    data = resp.json()
    data = data['records']['location']
    return data
    
def get_info(data) :
    row_data = []
    for item in data :
        lat = item['lat']
        lon = item['lon']
        locationName = item['locationName']
        stationId = item['stationId']
        obsTime = item['time']['obsTime']
        ELE = item['weatherElement'][0]['elementValue']
        WDIR = item['weatherElement'][1]['elementValue']
        WDSD = item['weatherElement'][2]['elementValue']
        TEMP = item['weatherElement'][3]['elementValue']
        HUMD = item['weatherElement'][4]['elementValue']
        PRES = item['weatherElement'][5]['elementValue']
        dic = {}
        dic['lat'] = lat
        dic['lon'] = lon
        dic['locationName'] = locationName
        dic['stationId'] = stationId
        dic['obsTime'] = obsTime
        dic['ELE'] = ELE
        dic['WDIR'] = WDIR
        dic['WDSD'] = WDSD
        dic['TEMP'] = TEMP
        dic['HUMD'] = HUMD
        dic['PRES'] = PRES
        row_data.append(dic)
    return row_data


def write_csv(row_data) :
    headers = ['lat','lon','locationName','stationId','obsTime','ELE','WDIR','WDSD','TEMP','HUMD','PRES']
    with open('weather.csv','w') as outfile :
        dic_writer = csv.DictWriter(outfile,headers)
        dic_writer.writeheader()
        dic_writer.writerows(row_data)

sched = BlockingScheduler({'apscheduler.timezone': 'Asia/Taipei'})

# decorator 設定 Scheduler 的類型和參數，例如 interval 間隔多久執行
@sched.scheduled_job('interval', seconds=20)
def timed_job():
    print('每20秒執行一次工作區塊')
    URL_api = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/O-A0001-001?Authorization=rdec-key-123-45678-011121314'   
    data = get_data(URL_api)
    row_data = get_info(data)
    write_csv(row_data)

# 開始執行
sched.start()